<br>

### Exercício: Regressão Linear:
__Parte 1:__

1- Usando a função getData(), carregue os dados disponibilizados.

2- Separe parte dos dados para o dataset de teste.

3- Usando a metodologia de validação cruzada, teste diferentes parâmetros da regLinear - diferentes learning_rates e num_steps - para escolher a melhor combinação de parâmetros.

4- Implemente a regressão linear do scikit-learn e compare os resultados obtidos.

__Parte 2 (Introdução):__

Para cada variável explicativa $X_1, .., X_5$, crie outras variáveis usando o __quadrado__ de cada um delas. Desta forma, o conjunto final será de 10 variáveis, em que:

$X_6 = (X_1)^{2}$, $X_7 = (X_2)^{2}$, $X_8 = (X_3)^{2}$, $X_9 = (X_4)^{2}$, $X_{10} = (X_5)^{2}$.

Ao treinarmos uma regressão linear com essas 10 variáveis, a predição é da forma:

$y_{pred} = \theta_0 + \theta_1 \cdot X_1 + .. + \theta_5 \cdot X_5 + \theta_6 \cdot (X_1)^{2} + .. + \theta_{10} \cdot (X_5)^{2}$

Como estamos usando o quadrado das variáveis explicativas, dizemos que temos um __modelo de regressão polinomial de grau 2__. Podemos ter variações deste modelo:

-Podemos aumentar o grau: basta mudar a potência que elevamos as variáveis. Por exemplo, podemos incluir o __cubo__ das variáveis e termos um modelo polinomial de ordem 3.

-Podemos ter __interações__ entre as variáveis: multiplicações entre as variáveis.

Exemplo:

$y_{pred} = \theta_0 + \theta_1 \cdot X_1 + .. + \theta_5 \cdot X_5 + \theta_6 \cdot (X_1)^{2} + .. + \theta_{10} \cdot (X_5)^{2} + \theta_{11} \cdot (X_1)^{3} + \theta_{12} \cdot V1 + \theta_{13} \cdot V2$,

onde

$V_1 = X_1 \cdot X_2$ e $V_2 = (X_2)^{2} \cdot X_4$

__Parte 2 (Exercício):__

1- Estude o link:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

em que é discutido como criar modelos polinomiais com o scikit-learn de forma detalhada.

2- Repita os passos da primeira parte, mas agora considerando polinômios de graus 2 ou mais.

3- Inclua regularização Ridge e Lasso nas análises e teste os resultados para diferentes parâmetros $\alpha$.

<br>

### Exercício: Regressão Logística:

__Parte 1:__

Crie uma classe regLogistica para treinar o modelo de regressão logística. Essa classe deve ser usada para problemas de classificação binária, cuja variável target assume os valores: 0 (classe negativa) e 1 (classe positiva).

O método construtor dessa classe deve possuir 3 parâmetros: learning_rate, num_steps e limiar.

Os outros médotos devem ser:

    - médoto fit: para treinar o modelo - usando gradient descent
    
    - médoto predict_proba: para retornar a probabilidade da classe 1
    
    - médoto predict: retornar a classe predita: 0 ou 1 - dependente do limiar
    
__Parte 2:__

Usando a função getData2(), carregue o dataset disponibilizado.

Use a regLogistica, classe criada na parte 1 do exercício, para treinar modelos nestes dados. Use validação cruzada para seleção dos parâmetros. Considere diferentes métricas de classificação e justifique as escolhas.

In [257]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_friedman1, make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import accuracy_score
import time

import warnings
warnings.filterwarnings("ignore")

### Exercício: Regressão Linear:
#### Parte 1:

In [2]:
#função para acessar os dados do exercício 1

def getData():
    X, y = make_friedman1(n_samples = 10000, n_features = 5, noise = 5, random_state = 0)
    return X, y

In [3]:
X,y = getData()

print(X[:10])

print('\n------------------------------------------------------------------\n')

print(y[:10].reshape(-1,1))

[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]
 [0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985 0.77815675 0.87001215]
 [0.97861834 0.79915856 0.46147936 0.78052918 0.11827443]
 [0.63992102 0.14335329 0.94466892 0.52184832 0.41466194]
 [0.26455561 0.77423369 0.45615033 0.56843395 0.0187898 ]
 [0.6176355  0.61209572 0.616934   0.94374808 0.6818203 ]
 [0.3595079  0.43703195 0.6976312  0.06022547 0.66676672]
 [0.67063787 0.21038256 0.1289263  0.31542835 0.36371077]]

------------------------------------------------------------------

[[15.28814034]
 [21.53563265]
 [15.33477388]
 [18.80973929]
 [16.17485067]
 [20.57416088]
 [ 6.39920803]
 [21.80022823]
 [ 3.90210375]
 [ 5.41205623]]


In [4]:
#Será separado 25% dos dados para teste

Xtrain, Xtest,ytrain,ytest = train_test_split(X,y, test_size = 0.25, random_state = 0)
print(X.shape,y.shape)
print(Xtrain.shape,ytrain.shape)
print(Xtest.shape,ytest.shape)

(10000, 5) (10000,)
(7500, 5) (7500,)
(2500, 5) (2500,)


In [5]:
#classe regLinear para exercício

class regLinear():
    
    def __init__(self, learning_rate, num_steps):
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        
    def fit(self, X, y):
        y = y.reshape(-1,1)
        m = X.shape[0] 
        k = X.shape[1] 
        theta = np.random.randn(k+1,1) 
        X_b = np.c_[np.ones((m, 1)), X] 
        for step in range(self.num_steps):
            gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
            theta = theta - self.learning_rate * gradients
        self.theta_final = theta
        #print("modelo treinado.")
        
    def predict(self, X):
        m = X.shape[0]
        X_b = np.c_[np.ones((m, 1)), X]
        preds = X_b.dot(self.theta_final)
        return preds.reshape(-1,)    

In [11]:
def regLinear_comparacao(num_steps,learning_rate,classificador_comparacao,Xtrain,ytrain,Xtest,ytest):
    
    num_steps=num_steps
    learning_rate=learning_rate
    classificador = classificador_comparacao
    classificador.fit(Xtrain, ytrain)
    classificador_predict_train = classificador.predict(Xtrain)
    classificador_predict_test = classificador.predict(Xtest)
    nome_classificador = str(classificador_comparacao).replace('()','')

    for num_steps, learning_rate in zip(num_steps,learning_rate):

        rg = regLinear(learning_rate = learning_rate, num_steps = num_steps)
        rg.fit(Xtrain, ytrain)
        rg_predict_train = rg.predict(Xtrain)
        rg_predict_test = rg.predict(Xtest)
        
        comparacao_treino = np.c_[np.round_(classificador_predict_train[:10],4), #Predições do classificador
                                  np.round_(rg_predict_train[:10],4), #Predições da regLinear
                                  np.round_(classificador_predict_train[:10],4)-np.round_(rg_predict_train[:10],4), #diff
                                  np.round_(ytrain[:10].reshape(-1,1),4) #Variável target
                                 ]
        
        RMSE_treino = np.sqrt(mean_squared_error(y_true = ytrain, y_pred = classificador_predict_train))
            
        comparacao_teste = np.c_[np.round_(classificador_predict_test[:10],4), 
                                 np.round_(rg_predict_test[:10],4),
                                 np.round_(classificador_predict_test[:10],4)-np.round_(rg_predict_test[:10],4),
                                 np.round_(ytest[:10].reshape(-1,1),4)
                                ]
        
        RMSE_teste = np.sqrt(mean_squared_error(y_true = ytest, y_pred = classificador_predict_test))
       

        print('Quantidade de passos:',num_steps)
        print('Taxa de aprendizado:',learning_rate)
        
        print('\nComparação dos resultados em dados de treino:')
        print(nome_classificador,'x reglinear x Diferença x Variável Target\n')
        print(comparacao_treino)
        print('\nRMSE dos dados de treino:',RMSE_treino)
        
        print('\n--------------------------------------------\n')
        
        print('\nComparação dos resultados em dados de teste:')
        print(nome_classificador,'x reglinear x Diferença x Variável Target\n')
        print(comparacao_teste)
        print('\nRMSE dos dados de teste:',RMSE_teste,'\n')

In [12]:
num_steps=[250,500,750,1000]
learning_rate=[0.1, 0.2,0.3, 0.4]

regLinear_comparacao(num_steps=num_steps,
                     learning_rate=learning_rate,
                     classificador_comparacao=LinearRegression(),
                     Xtrain=Xtrain,
                     ytrain=ytrain,
                     Xtest=Xtest,
                     ytest=ytest)

Quantidade de passos: 250
Taxa de aprendizado: 0.1

Comparação dos resultados em dados de treino:
LinearRegression x reglinear x Diferença x Variável Target

[[ 1.94989e+01  1.93850e+01  1.13900e-01  1.28066e+01]
 [ 1.12942e+01  1.12800e+01  1.42000e-02  1.32577e+01]
 [ 1.08702e+01  1.10888e+01 -2.18600e-01  1.25476e+01]
 [ 1.78540e+01  1.77759e+01  7.81000e-02  1.84473e+01]
 [ 9.64440e+00  1.00951e+01 -4.50700e-01 -1.77690e+00]
 [ 8.14170e+00  8.48790e+00 -3.46200e-01  4.01050e+00]
 [ 1.33002e+01  1.35038e+01 -2.03600e-01  1.83020e+01]
 [ 7.86290e+00  8.16790e+00 -3.05000e-01  1.49591e+01]
 [ 1.10141e+01  1.11852e+01 -1.71100e-01  6.50070e+00]
 [ 1.68789e+01  1.70013e+01 -1.22400e-01  2.17621e+01]]

RMSE dos dados de treino: 5.5527435100835625

--------------------------------------------


Comparação dos resultados em dados de teste:
LinearRegression x reglinear x Diferença x Variável Target

[[ 1.61365e+01  1.61335e+01  3.00000e-03  1.69377e+01]
 [ 1.33709e+01  1.33919e+01 -2.10000e

### A regLinear se comportou bem melhor com step a partir de 750 e taxa de aprendizado a partir de 0,3. Nesse contexto, podemos testar step = 1000 com uma taxa menor (0,1) e ver como o modelo se comporta. O RMSE também ficou alinhado com o parâmetro noise=5.0 do make_friedman1.

In [13]:
num_steps=[1000]
learning_rate=[0.1]

regLinear_comparacao(num_steps=num_steps,
                     learning_rate=learning_rate,
                     classificador_comparacao=LinearRegression(),
                     Xtrain=Xtrain,
                     ytrain=ytrain,
                     Xtest=Xtest,
                     ytest=ytest)

Quantidade de passos: 1000
Taxa de aprendizado: 0.1

Comparação dos resultados em dados de treino:
LinearRegression x reglinear x Diferença x Variável Target

[[ 1.94989e+01  1.94987e+01  2.00000e-04  1.28066e+01]
 [ 1.12942e+01  1.12938e+01  4.00000e-04  1.32577e+01]
 [ 1.08702e+01  1.08710e+01 -8.00000e-04  1.25476e+01]
 [ 1.78540e+01  1.78538e+01  2.00000e-04  1.84473e+01]
 [ 9.64440e+00  9.64620e+00 -1.80000e-03 -1.77690e+00]
 [ 8.14170e+00  8.14280e+00 -1.10000e-03  4.01050e+00]
 [ 1.33002e+01  1.33011e+01 -9.00000e-04  1.83020e+01]
 [ 7.86290e+00  7.86390e+00 -1.00000e-03  1.49591e+01]
 [ 1.10141e+01  1.10147e+01 -6.00000e-04  6.50070e+00]
 [ 1.68789e+01  1.68796e+01 -7.00000e-04  2.17621e+01]]

RMSE dos dados de treino: 5.5527435100835625

--------------------------------------------


Comparação dos resultados em dados de teste:
LinearRegression x reglinear x Diferença x Variável Target

[[ 1.61365e+01  1.61366e+01 -1.00000e-04  1.69377e+01]
 [ 1.33709e+01  1.33710e+01 -1.00000

### Os resultados da regLinear foram praticamente idênticos ao da regressão logística com os parâmetros acima!!!

In [14]:
cv_lin_reg = cross_val_score(estimator = LinearRegression(), 
                             X = Xtrain, 
                             y = ytrain, 
                             cv = 5, 
                             scoring = 'r2')
print(cv_lin_reg)
print('media: ', cv_lin_reg.mean())
print('desvio padrão: ', cv_lin_reg.std())

[0.36453906 0.37397915 0.3635744  0.38055211 0.37465289]
media:  0.3714595229612249
desvio padrão:  0.006469764437639058


### Exercício: Regressão Linear:
#### Parte 2 - Regressão polinomial:

In [32]:
def polyFit(X, y, grau, base_model,base_model_name):

    for grau in grau:
        
        polybig_features = PolynomialFeatures(degree = grau, include_bias = False)
        std_scaler = StandardScaler()
        basemodel = base_model
    
        #criando um pipeline: sequencia de execução de passos
        polynomial_regression = Pipeline([
                ("poly_features", polybig_features),
                ("std_scaler", std_scaler),
                ("base_model_name", base_model),
            ])
    
        polyFit = polynomial_regression.fit(X, y)
        
        print("Regularização:",Model_name)
    
        print("grau: ", grau)

        polyFit.predict(X)

        print("RMSE:")
        print(np.sqrt(np.mean(np.square(y - polyFit.predict(X)))))
        print("------------------------------------------------")

#### Regularização Ridge em dados de treino

In [33]:
grau = [2,5,10]
Model_name= 'Ridge'

t0 = time.time()

for alpha in [0.1,0.01,0.001]:
    
    print("Alpha:",alpha)
    polyFit(X = Xtrain,
            y = ytrain,
            grau = grau,
            base_model = Ridge(alpha = alpha),
            base_model_name = Model_name
       )

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

Alpha: 0.1
Regularização: Ridge
grau:  2
RMSE:
5.158800384287972
------------------------------------------------
Regularização: Ridge
grau:  5
RMSE:
4.903875114891525
------------------------------------------------
Regularização: Ridge
grau:  10
RMSE:
4.6995005165153945
------------------------------------------------
Alpha: 0.01
Regularização: Ridge
grau:  2
RMSE:
5.1588002758506075
------------------------------------------------
Regularização: Ridge
grau:  5
RMSE:
4.896345642636893
------------------------------------------------
Regularização: Ridge
grau:  10
RMSE:
4.609064755944324
------------------------------------------------
Alpha: 0.001
Regularização: Ridge
grau:  2
RMSE:
5.158800274765201
------------------------------------------------
Regularização: Ridge
grau:  5
RMSE:
4.895380215627292
------------------------------------------------
Regularização: Ridge
grau:  10
RMSE:
4.501813372896098
------------------------------------------------
tempo (em segundos) para execuçã

#### Regularização Ridge em dados de teste

In [34]:
grau = [2,5,10]
Model_name= 'Ridge'

t0 = time.time()

for alpha in [0.1,0.01,0.001]:
    
    print("Alpha:",alpha)
    polyFit(X = Xtest,
            y = ytest,
            grau = grau,
            base_model = Ridge(alpha = alpha),
            base_model_name = Model_name
       )

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

Alpha: 0.1
Regularização: Ridge
grau:  2
RMSE:
5.155017888946525
------------------------------------------------
Regularização: Ridge
grau:  5
RMSE:
4.8308845584602915
------------------------------------------------
Regularização: Ridge
grau:  10
RMSE:
4.337394958967313
------------------------------------------------
Alpha: 0.01
Regularização: Ridge
grau:  2
RMSE:
5.1550168876582845
------------------------------------------------
Regularização: Ridge
grau:  5
RMSE:
4.7953502841510005
------------------------------------------------
Regularização: Ridge
grau:  10
RMSE:
4.088903782383004
------------------------------------------------
Alpha: 0.001
Regularização: Ridge
grau:  2
RMSE:
5.155016877615181
------------------------------------------------
Regularização: Ridge
grau:  5
RMSE:
4.783730451422006
------------------------------------------------
Regularização: Ridge
grau:  10
RMSE:
3.7851623331572615
------------------------------------------------
tempo (em segundos) para execu

#### Regularização Lasso em dados de treino

In [35]:
grau = [2,5,10]
Model_name= 'Lasso'

t0 = time.time()

for alpha in [0.1,0.01,0.001]:
    
    print("Alpha:",alpha)
    polyFit(X = Xtrain,
            y = ytrain,
            grau = grau,
            base_model = Lasso(alpha = alpha),
            base_model_name = Model_name
       )

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

Alpha: 0.1
Regularização: Lasso
grau:  2
RMSE:
5.347414476589604
------------------------------------------------
Regularização: Lasso
grau:  5
RMSE:
5.047841298242992
------------------------------------------------
Regularização: Lasso
grau:  10
RMSE:
5.055161150236008
------------------------------------------------
Alpha: 0.01
Regularização: Lasso
grau:  2
RMSE:
5.161218564323897
------------------------------------------------
Regularização: Lasso
grau:  5
RMSE:
4.9687394630443755
------------------------------------------------
Regularização: Lasso
grau:  10
RMSE:
4.946128240377529
------------------------------------------------
Alpha: 0.001
Regularização: Lasso
grau:  2
RMSE:
5.158838965255326
------------------------------------------------
Regularização: Lasso
grau:  5
RMSE:
4.9381121242394395
------------------------------------------------
Regularização: Lasso
grau:  10
RMSE:
4.876753769114236
------------------------------------------------
tempo (em segundos) para execuçã

#### Regularização Ridge em dados de teste

In [36]:
grau = [2,5,10]
Model_name= 'Lasso'

t0 = time.time()

for alpha in [0.1,0.01,0.001]:
    
    print("Alpha:",alpha)
    polyFit(X = Xtest,
            y = ytest,
            grau = grau,
            base_model = Lasso(alpha = alpha),
            base_model_name = Model_name
       )

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

Alpha: 0.1
Regularização: Lasso
grau:  2
RMSE:
5.34874980218386
------------------------------------------------
Regularização: Lasso
grau:  5
RMSE:
5.074760764493081
------------------------------------------------
Regularização: Lasso
grau:  10
RMSE:
5.066938193727527
------------------------------------------------
Alpha: 0.01
Regularização: Lasso
grau:  2
RMSE:
5.158047115766565
------------------------------------------------
Regularização: Lasso
grau:  5
RMSE:
4.964678114797044
------------------------------------------------
Regularização: Lasso
grau:  10
RMSE:
4.895090229976122
------------------------------------------------
Alpha: 0.001
Regularização: Lasso
grau:  2
RMSE:
5.155054267158118
------------------------------------------------
Regularização: Lasso
grau:  5
RMSE:
4.905463617081603
------------------------------------------------
Regularização: Lasso
grau:  10
RMSE:
4.6950685004706685
------------------------------------------------
tempo (em segundos) para execução:

### Exercício: Regressão Logística:
#### Parte 1 - Criar uma classe para treinar dados em regressão logística::

In [333]:
class regLogistica():
    
    def __init__(self, learning_rate, num_steps, limiar):
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        self.limiar = limiar
        
    def sigmoid(self, t):
        return 1 / (1 + np.exp(-t))
       
    def fit(self, X, y):
        
        X_b = np.c_[np.ones(X.shape[0]), X]
        theta = np.random.randn(X_b.shape[1],1) #instanciando um vetor de parâmetros theta aleatório
        
        for step in range(self.num_steps):

            #calculando a probabilidade 
            yscores = sigmoid(X_b.dot(theta))

            #calculando o gradiente da Log Loss
            gradient = X_b.T.dot(yscores - y)

            #atualizando os pesos
            theta = theta - self.learning_rate * gradient

        self.theta_final = theta
              
    def predict_proba(self,theta_final):
        t = X_b.dot(theta_final)
        yscores = sigmoid(t)
        ypred = np.where(yscores > limiar, 1, 0)
        return ypred

### Exercício: Regressão Logística:
#### Parte 2 - Usar a classe criada acima para treinar os dados carregados abaixo:

In [334]:
#função para acessar os dados do exercício 2

def getData2():
    X, y = make_classification(n_classes=2, n_features=5, n_samples=10000, random_state = 0)
    y= y.reshape(-1,1)
    return X, y

In [335]:
X, y = getData2()

print(X[:10])

print('\n------------------------------------------------------------------\n')

print(y[:10])

[[-0.82380715 -0.59163837  0.13041933 -0.40345475  1.16360785]
 [ 0.7091986   0.60606127 -0.37678226  0.39654936 -1.15961369]
 [ 1.61194498  0.36486859  1.91264129  0.38601731 -0.31972146]
 [-0.87521758  0.03070064 -1.66414467 -0.09315833  0.30731465]
 [-1.32532555  0.19953765 -2.9384829  -0.06318708 -0.96281507]
 [-0.97739632 -1.14675541  1.37104493 -0.70502371 -0.07881854]
 [-0.38160656 -1.07066645  2.23867692 -0.59225403 -1.58645395]
 [ 1.3759365   1.29166811 -1.04773872  0.8282992  -0.61000978]
 [ 1.29969415  0.56763584  0.79442277  0.45038871 -1.26101667]
 [ 0.43890899  0.55569056 -0.72705201  0.3373232  -0.55876158]]

------------------------------------------------------------------

[[0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]]


In [336]:
Xtrain, Xtest,ytrain,ytest = train_test_split(X,y, test_size = 0.25, random_state = 0)
print(X.shape,y.shape)
print(Xtrain.shape,ytrain.shape)
print(Xtest.shape,ytest.shape)

(10000, 5) (10000, 1)
(7500, 5) (7500, 1)
(2500, 5) (2500, 1)


In [337]:
limiares = np.linspace(0.05,0.95,19)

for limiar in limiares:
    rL = regLogistica(learning_rate=0.01,num_steps=100, limiar=limiar)
    rL.fit(X=Xtrain, y=ytrain)
    ypred = rL.predict_proba(theta_final)
    
    print("Limiar:",limiar)
    print("Assertividade da regLogistica:",str(round(((ypred == ytrain).sum() / ypred.size)*100,2))+'%\n')

Limiar: 0.05
Assertividade da regLogistica: 82.49%

Limiar: 0.1
Assertividade da regLogistica: 82.47%

Limiar: 0.15
Assertividade da regLogistica: 82.51%

Limiar: 0.2
Assertividade da regLogistica: 82.48%

Limiar: 0.25
Assertividade da regLogistica: 82.41%

Limiar: 0.3
Assertividade da regLogistica: 82.4%

Limiar: 0.35
Assertividade da regLogistica: 82.36%

Limiar: 0.39999999999999997
Assertividade da regLogistica: 82.39%

Limiar: 0.44999999999999996
Assertividade da regLogistica: 82.37%

Limiar: 0.49999999999999994
Assertividade da regLogistica: 82.43%

Limiar: 0.5499999999999999
Assertividade da regLogistica: 82.44%

Limiar: 0.6
Assertividade da regLogistica: 82.45%

Limiar: 0.65
Assertividade da regLogistica: 82.47%

Limiar: 0.7
Assertividade da regLogistica: 82.51%

Limiar: 0.75
Assertividade da regLogistica: 82.49%

Limiar: 0.7999999999999999
Assertividade da regLogistica: 82.45%

Limiar: 0.85
Assertividade da regLogistica: 82.35%

Limiar: 0.9
Assertividade da regLogistica: 82.35%

### Métricas de classificação

#### Comparação da classe regLogística com a Logistic Regression do SckitLearn

#### Vimos que o comportamento dos resultados foi bem parecido ao variar os limiares, então para compararmos os resultados com o SckitLearn vamos manter o limiar padrão de 0,5.

In [342]:
logreg = LogisticRegression()
logreg = logreg.fit(Xtrain, ytrain)
logreg = logreg.predict(Xtrain).reshape(-1,1)

rL = regLogistica(learning_rate=0.01,num_steps=100, limiar=0.5)
rL.fit(X=Xtrain, y=ytrain)
ypred = rL.predict_proba(theta_final)

In [343]:
print("Assertividade da regLogistica:",str(round(((ypred == ytrain).sum() / ypred.size)*100,2))+'%')
print("Assertividade da Logistic Regression do Sklearn:", str(round(((logreg == ytrain).sum() / logreg.size)*100,2))+'%')

Assertividade da regLogistica: 82.31%
Assertividade da Logistic Regression do Sklearn: 82.56%


#### Acurácia

In [340]:
accuracy_score(y_pred = ytrain, y_true = ypred)

0.8230666666666666

#### Matriz de confusão

In [341]:
from sklearn.metrics import confusion_matrix

print("Verdadeiros positivos:",confusion_matrix(y_true = ytrain, y_pred = ypred)[0,0])
print("Falsos positivos:",confusion_matrix(y_true = ytrain, y_pred = ypred)[0,1])
print("Falsos negativos:",confusion_matrix(y_true = ytrain, y_pred = ypred)[1,0])
print("Verdadeiros negativos:",confusion_matrix(y_true = ytrain, y_pred = ypred)[1,1])

Verdadeiros positivos: 3163
Falsos positivos: 604
Falsos negativos: 723
Verdadeiros negativos: 3010


#### Justificativa

#### Vimos em aula que somente a acurácia nem sempre é um bom sistema de classificação se olhada de forma isolada. Nesse contexto, a matriz de confusão é um reforço para a análise e comprovou a eficácia demonstrada pelo modelo através da acurácia.